# Shipwrecked

In this project, we will write a Python program that uses Bayes' Rule to quickly find a solitary fisherman who has gone missing. As the director of the Coast Guard's search and rescue operations for the region, we've already interviewed the fisherman's wife and <b>determined his last known position</b>, now more than six hours old.
He radioed that he was abandoning ship, but no one knows if he's in a life raft or floating in the sea. The waters around the cape are warm, but if he's immersed, he'll experience hypothermia in approximately 12 hours. If he's wearing a floatation device and is lucky, he might last three days.

### The Objective
We will create a search and rescue game that uses Bayes' Rule to inform player choices on how to conduct a search.

### The Strategy
We'll start with initial target probabilities for the sailor's location and update them for the search results. If we achieve an effective search of an area but find nothing, the probability that the sailor is in another area will increase.
But, just as in real life, there are two ways things could go wrong: you thoroughly search an area but still miss the sailor, or your search goes poorly, wasting a day's effort. To equate this to search effectiveness scores, in the first case, you might get an SEP of 0.85, but the sailor is in the remaining 15% of the area not searched. In the second case, your SEP is 0.2, and you've left 80 percent of the area unsearched.

To aid the player, we will use the OpenCV library to build an interface for working with the program. Although the interface can be something simple like a menu built in the shell, you'll also want a map of the cape and the search areas. The OpenCV library is an excellent choice for this game, since it lets you display images and add drawings and text.

In [ ]:
#importing the necessary packages
import sys, random, itertools, numpy as np, cv2 as cv

MAP_FILE = 'cape_python.png'

SA1_CORNERS = (130, 265, 180, 315) #(UL-X, UL-Y, LR-X, LR-Y)
SA2_CORNERS = (80, 255, 130, 305)
SA3_CORNERS = (105, 205, 155, 255)

#Defining the Search Class
class Search():
    """Bayesian Search & Rescue game """

    def __init__(self, name):
        self.name = name
        self.img = cv.imread(MAP_FILE, cv.IMREAD_COLOR)
        if self.img is None:
            print("Could not load map file {}".format(MAP_FILE), file=sys.stderr)
            sys.exit(1)
        
        self.area_actual = 0
        self.sailor_actual = [0, 0] #As "local" coords within search area

        self.sa1 = self.img[SA1_CORNERS[1] : SA1_CORNERS[3], SA1_CORNERS[0] : SA1_CORNERS[2]]
        self.sa2 = self.img[SA2_CORNERS[1] : SA2_CORNERS[3], SA2_CORNERS[0] : SA2_CORNERS[2]]
        self.sa3 = self.img[SA3_CORNERS[1] : SA3_CORNERS[3], SA3_CORNERS[0] : SA3_CORNERS[2]]

        self.p1 = 0.2
        self.p2 = 0.5
        self.p3 = 0.3

        self.sep1 = 0
        self.sep2 = 0
        self.sep3 = 0
    
    #Defining a function to draw the base map
    def draw_map(self, last_known):
        """Display basemap with scale, last known xy location, search areas"""
        cv.line(self.img, (20, 370), (70, 370), (0, 0, 0), 2)
        cv.putText(self.img, '0', (8, 370), cv.FONT_HERSEY_PLAIN, 1, (0, 0, 0))
        cv.putText(self.img, '50 Nautical Miles', (71, 370), cv.FONT_HERSEY_PLAIN, 1, (0, 0, 0))

        cv.rectangle(self.img, (SA1_CORNERS[0], SA1_CORNERS[1]), (SA1_CORNERS[2], SA1_CORNERS[3]), (0, 0, 0), 1)
        cv.putText(self.img, '1', (SA1_CORNERS[0] + 3, SA1_CORNERS[1] + 15), cv.FONT_HERSHEY_PLAIN, 1, 0)

        cv.rectangle(self.img, (SA2_CORNERS[0], SA2_CORNERS[1]), (SA2_CORNERS[2], SA2_CORNERS[3]), (0, 0, 0), 1)
        cv.putText(self.img, '2', (SA2_CORNERS[0] + 3, SA2_CORNERS[1] + 15), cv.FONT_HERSHEY_PLAIN, 1, 0)

        cv.rectangle(self.img, (SA3_CORNERS[0], SA3_CORNERS[1]), (SA3_CORNERS[2], SA3_CORNERS[3]), (0, 0, 0), 1)
        cv.putText(self.img, '3', (SA3_CORNERS[0] + 3, SA3_CORNERS[1] + 15), cv.FONT_HERSHEY_PLAIN, 1, 0)

        cv.putText(self.img, '+', (last_known), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
        cv.putText(self.img, '+ = Last Known Position', (274, 355), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
        cv.putText(self.img, '* = Actual Position', (275, 370), cv.FONT_HERSHEY_PLAIN, 1, (255, 0, 0))

        cv.imshow('Search Area', self.img)
        cv.moveWindow('Search Area', 750, 10)
        cv.waitKey(500)
    
    #defining a function to randomly choose the sailor's actual location
    def sailor_final_location(self, num_search_areas):
        """Return the actual x,y location of the missing sailor"""
        #Find sailor coordinates with respect to any Search Area subarray
        self.sailor_actual[0] = np.random.choice(self.sa1.shape[1], 1)
        self.sailor_actual[1] = np.random.choice(self.sa1.shape[0], 1)

        area = int(random.triangular(1, num_search_areas + 1))

        if area == 1:
            x = self.sailor_actual[0] + SA1_CORNERS[0]
            y = self.sailor_actual[1] + SA1_CORNERS[1]
            self.area_actual = 1
        elif area == 2:
            x = self.sailor_actual[0] + SA1_CORNERS[0]
            y = self.sailor_actual[1] + SA2_CORNERS[1]
            self.area_actual = 2
        elif area == 3:
            x = self.sailor_actual[0] + SA3_CORNERS[0]
            y = self.sailor_actual[1] + SA3_CORNERS[1]
            self.actual_actual = 3
        return x, y

In real life, the sailor would drift along, and the odds of their moving into area 3 would increase with each search. For this game, we kept this a static location, to make the logic behind Bayes' rule as clear as possible. As a result, this scenario behaves more like a search for a sunken submarine.

In real life, weather and mechanical problems can result in low search effectiveness scores. Thus, the strategy for each search will be to generate a list of all possible locations within a search area, shuffle the list, and then sample it based on the search effectiveness value. Because the SEP will never be 1.0, if you just sample from the start or end of the list--without shuffling-- you'll never be able to access coordinates tucked away in its "tail".

In [ ]:
#Defining methods to randomly choose search effectiveness and conduct search
def calc_search_effectiveness(self):
    """Set decimal search effectiveness value per search area."""
    self.sep1 = random.uniform(0.2, 0.9)
    self.sep2 = random.uniform(0.2, 0.9)
    self.sep3 = random.uniform(0.2, 0.9)

def conduct_search(self, area_num, area_array, effectiveness_prob):
    """Return search results and list of searched coordinates."""
    local_y_range = range(area_array.shape[0])
    local_x_range = range(area_array.shape[1])
    coords = list(itertools.product(local_x_range, local_y_range))
    random.shuffle(coords)
    coords = coords[:int((len(coords) * effectiveness_prob))]
    loc_actual = (self.sailor_actual[0], self.sailor_actual[1])
    if area_num == self.area_actual and loc_actual in coords:
        return 'Found in Area {}.'.format(area_num), coords
    else:
        return 'Not Found', coords